## Motion Planning through KOMO ##

In [2]:
import robotic as ry
import numpy as np
import time

## Part a) Adding a constraint to simulation

In [12]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

# add waypoints
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way2'). setShape(ry.ST.marker, [.1]) .setPosition([0, .2, 1.6])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

# add wall
C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()

# save home joints state
qHome = C.getJointState()

First, solve the task without collision constraint

In [13]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)

# order=0: penalizing sqr distance to qHome, 
# order=1: penalizing sqr distances between consecutive configurations (velocities), 
# order=2: penalizing accelerations across 3 configurations
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)

# add accumuated colisions penalty
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)


# add waypoints as objectives
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', 'way2'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([4], ry.FS.positionDiff, ['l_gripper', 'way2'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

# get the initial path using Non Linear Programming solver
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)

# get the optimized path. q is a matrix of joint states
q = komo.getPath()
print(q)

# simulate the optimized path
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.537728, evals: 237, done: 1, feasible: 1, sos: 5.64877, f: 0, ineq: 0, eq: 0.146168 }
[[-0.03415093 -0.44958748 -0.03992019 -2.00091532 -0.01265128  2.0063354
  -0.49757908]
 [-0.06832475 -0.39911534 -0.07989052 -2.00181758 -0.02531657  2.012656
  -0.49515019]
 [-0.1025443  -0.34852387 -0.11996122 -2.00269368 -0.03800998  2.01894695
  -0.49270535]
 [-0.13683234 -0.29775321 -0.16018271 -2.00353043 -0.05074587  2.02519328
  -0.49023656]
 [-0.17121146 -0.24674322 -0.20060571 -2.00431454 -0.06353891  2.03137981
  -0.4877358 ]
 [-0.20570401 -0.19543344 -0.24128136 -2.00503247 -0.07640427  2.03749107
  -0.48519498]
 [-0.24033207 -0.14376292 -0.28226129 -2.00567045 -0.08935778  2.04351117
  -0.48260602]
 [-0.27511729 -0.09167013 -0.32359778 -2.00621429 -0.10241614  2.04942361
  -0.47996074]
 [-0.31008078 -0.03909279 -0.36534391 -2.00664929 -0.11559714  2.05521109
  -0.47725093]
 [-0.34524295  0.0140323  -0.40755368 -2.00696005 -0.12891999  2.06085522
  -0.47446833]
 [-0.29965399 -0.

In [14]:
del komo
del C

Then, we add collision constraint with "accumulatedCollisions" feature

In [15]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [19]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.279496, evals: 164, done: 1, feasible: 1, sos: 0.104906, f: 0, ineq: 0, eq: 0.113958 }
[[-3.56298038e-03 -4.95615014e-01 -2.82160093e-03 -1.99974272e+00
  -2.23833732e-03  2.00081006e+00 -5.00076435e-01]
 [-8.51536114e-03 -4.89251649e-01 -7.35003858e-03 -1.99989926e+00
  -4.71259846e-03  2.00217752e+00 -5.00230075e-01]
 [-1.66515252e-02 -4.78546468e-01 -1.56791559e-02 -2.00092067e+00
  -7.79456562e-03  2.00484362e+00 -5.00532526e-01]
 [-3.04441614e-02 -4.60429537e-01 -3.05272884e-02 -2.00331247e+00
  -1.20903656e-02  2.00985309e+00 -5.01027714e-01]
 [-5.30476384e-02 -4.31025595e-01 -5.51809549e-02 -2.00759990e+00
  -1.84402709e-02  2.01855079e+00 -5.01702123e-01]
 [-8.78970273e-02 -3.86040342e-01 -9.30174594e-02 -2.01421969e+00
  -2.77577094e-02  2.03239567e+00 -5.02475047e-01]
 [-1.37831684e-01 -3.21848222e-01 -1.46559977e-01 -2.02336284e+00
  -4.07038143e-02  2.05254948e+00 -5.03226528e-01]
 [-2.03866542e-01 -2.37173324e-01 -2.16237468e-01 -2.03483902e+00
  -5.73053175e-02 

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [20]:
del komo
del C

In [28]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.0])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [29]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)

komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])

komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.168167, evals: 373, done: 1, feasible: 1, sos: 61.2207, f: 0, ineq: 0, eq: 0.11978 }
size of path: (10, 7)


## Possible State Solution by RRT

In [30]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [31]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_panda_coll5) [64,51] 	d=-0
{ time: 0.0055, evals: 77, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [32]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [33]:
del komo
del rrt
del C

Now, let us give a state which is impossible for robot arm to reach.

##  Part c) Impossible State Solution by KOMO

In [34]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [35]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.142184, evals: 292, done: 1, feasible: 0, sos: 28.8058, f: 0, ineq: 0, eq: 2.87005 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [36]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [37]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
{ time: 0.166388, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }


So we will get an error since there is no feasible solution

In [38]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range

In [41]:
# https://marctoussaint.github.io/robotics-course/tutorials.html -> Tutorials
# https://github.com/MarcToussaint/rai-tutorials/tree/main -> Tutorials
# https://marctoussaint.github.io/robotics-course/rai.html -> Documentation